In [ ]:
import uuid
import requests
import json
import time
import logging
from datetime import datetime
from kafka import KafkaProducer

def get_data():
    try:
        res = requests.get("https://randomuser.me/api/")
        res.raise_for_status()
        return res.json()['results'][0]
    except requests.exceptions.RequestException as e:
        logging.error(f"An error occurred while fetching data: {e}")
        return None

def format_data(res):
    try:
        data = {}
        location = res['location']
        data['id'] = str(uuid.uuid4())
        data['first_name'] = res['name']['first']
        data['last_name'] = res['name']['last']
        data['gender'] = res['gender']
        data['address'] = f"{location['street']['number']} {location['street']['name']}, {location['city']}, {location['state']}, {location['country']}"
        data['post_code'] = location['postcode']
        data['email'] = res['email']
        data['username'] = res['login']['username']
        data['dob'] = res['dob']['date']
        data['registered_date'] = res['registered']['date']
        data['phone'] = res['phone']
        data['picture'] = res['picture']['medium']
        return data
    except KeyError as e:
        logging.error(f"Error in formatting data: {e}")
        return None

def stream_data():
    try:
        producer = KafkaProducer(bootstrap_servers=['broker:29092'], max_block_ms=5000)
        curr_time = time.time()

        while True:
            if time.time() > curr_time + 60: # 1 minute
                break
            try:
                res = get_data()
                if res:
                    formatted_data = format_data(res)
                    if formatted_data:
                        producer.send('users_info', json.dumps(formatted_data).encode('utf-8'))
            except Exception as e:
                logging.error(f'An error occurred: {e}')
                continue
    except Exception as e:
        logging.error(f"An unexpected error occurred: {e}")

def main():
    logging.basicConfig(filename='/path/to/logfile.log', level=logging.ERROR, format='%(asctime)s - %(levelname)s - %(message)s')
    stream_data()

if __name__ == "__main__":
    main()